In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


fname = 'TCMD_2'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


df = pd.read_csv('datasets/Tweet_Classification_Moroccan_Dataset/data.csv', encoding='utf-16', engine='python', sep='\t') #, quotechar="'"  , quoting=3

display(len(df))
      
display(df.columns)
display(df[:4])



c = df['Topic'].value_counts()
display(c)

classes = set(df['Topic'].values)
display(classes)

df['Topic'] = df['Topic'].astype('category')
df['label'] = df['Topic'].cat.codes

df = df[['Tweet', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))


max_sequence_length = 128



models = [ 
        'faisalq/EgyBERT',            
    'faisalq/SaudiBERT',            
    'tunis-ai/TunBERT',
    'alger-ia/dziribert',
    'SI2M-Lab/DarijaBERT',
    'otmangi/MorRoBERTa',
    'otmangi/MorrBERT'
            
]


seeds = [0, 1, 42]

for model_name in models:
    for seed in seeds:
        ds = Dataset.from_pandas(df)
        ds = ds.train_test_split(test_size=0.2, seed = seed)
        if seed==0:
            display(ds)
            
        for i in range(3):
            print(f'{model_name}, try:{i}')
                  
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=classes_num).to('cuda')                                                 
            dataset_train = ds['train']
            dataset_validation = ds['test']                                                    
            
          
    
            def preprocess_function(examples):
                return tokenizer(examples['Tweet'], truncation=True, padding="max_length",
                                max_length=max_sequence_length)
            
            
            dataset_train = dataset_train.map(preprocess_function, batched=True)
            dataset_validation = dataset_validation.map(preprocess_function, batched=True)
            
           
            
            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)    
                acc = accuracy_score(labels, predictions)        
                f1 = f1_score(labels, predictions, average='macro')   
                with open(log_file, 'a') as f:
                    f.write(f'{model_name},{acc},{f1}\n')
                return {'accuracy': acc, 'f1_score': f1}
    
    
            
            
            epochs = 20
            save_steps = 10000 #save checkpoint every 10000 steps
            batch_size = 64
            
            training_args = TrainingArguments(
                output_dir = 'bert/',
                overwrite_output_dir=True,
                num_train_epochs = epochs,
                per_device_train_batch_size = batch_size,
                per_device_eval_batch_size = batch_size,
                save_steps = save_steps,
                save_total_limit = 1, #only save the last 5 checkpoints
                fp16=True,
                learning_rate = 5e-5,  # 5e-5 is the default
                logging_steps = 25, #50_000
                evaluation_strategy = 'steps',
                # evaluate_during_training = True,
                eval_steps = 25
                
            )
            
            trainer = Trainer(
                model = model,
                args = training_args,
                # data_collator=data_collator,
                train_dataset=dataset_train,
                eval_dataset=dataset_validation,
                compute_metrics = compute_metrics
            )
            
            
            trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



2024-09-18 20:11:13.035247: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-18 20:11:13.058838: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-18 20:11:13.442265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2399

Index(['Tweet', 'Topic'], dtype='object')

,Tweet,Topic
0,برا وخيط: أمل بنكيران... مصممة تبتكر الزي المغربي التقليدي بلمسة عالمية,ميديا
1,ربورتاج: قافلة توجيه مُتنقّلة تُطلع تلاميذ العيون على مختلف الآفاق الدراسية,مجتمع
2,ساعة الفطور: طريقة تحضير «وراب» بالخضر وصلصة «تزاتزيكي »,ميديا
3,ربورتاج: مائدة إفطار رمضانية تجمع الديانات الثلاث بالدار البيضاء,ثقافة


Topic
مجتمع     357
رياضة     354
ميديا     353
دولي      348
اقتصاد    344
سياسة     333
ثقافة     310
Name: count, dtype: int64

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at otmangi/MorrBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1919 [00:00<?, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
25,1.479900,1.155702,0.614583,0.614182
50,0.822500,0.963618,0.675000,0.678448
75,0.465300,0.983825,0.714583,0.718559
100,0.234900,1.044207,0.700000,0.704417
125,0.123300,1.202165,0.697917,0.700769
150,0.066300,1.241941,0.695833,0.699047
175,0.030400,1.366338,0.700000,0.703801
200,0.050900,1.440182,0.691667,0.696254
225,0.016300,1.541231,0.693750,0.699595
250,0.010600,1.551683,0.689583,0.695911


,Model,Accuracy,F1
0,SI2M-Lab/DarijaBERT,0.795833,0.793818
3,alger-ia/dziribert,0.816667,0.817497
9,faisalq/EgyBERT,0.822917,0.823959
11,faisalq/SaudiBERT,0.835417,0.834487
14,otmangi/MorRoBERTa,0.750000,0.753630
17,otmangi/MorrBERT,0.752083,0.748821
20,tunis-ai/TunBERT,0.414583,0.415796
